# Face-Recognition
Este notebook tem como objetivo fornecer um meio de avaliar os modelos presentes no Face-Recognition utilizando o dataset LFW.

## Importações e Inicialização

In [ ]:
import sys
import os
import torch
import numpy as np

sys.path.append('../../src/models/face-recognition')
from evaluate import eval, compute_metrics_from_predictions, compute_roc_metrics, compute_confusion_matrix
from models import (
    sphere20,
    sphere36,
    sphere64,
    MobileNetV1,
    MobileNetV2,
    mobilenet_v3_small,
    mobilenet_v3_large
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Selecionar Modelo e Checkpoint

In [2]:
# Modelos Disponíveis
available_models = {
    "sphere20": sphere20,
    "sphere36": sphere36,
    "sphere64": sphere64,
    "mobilenetv1": MobileNetV1,
    "mobilenetv2": MobileNetV2,
    "mobilenetv3_small": mobilenet_v3_small,
    "mobilenetv3_large": mobilenet_v3_large
}

print("Available models:")
for i, model_name in enumerate(available_models.keys(), 1):
    print(f"{i}. {model_name}")

# Selecionar peso
model_name = "mobilenetv3_large"
checkpoint_name = "mobilenetv3_large_5"
embedding_dim = 512
model_path = f"../../src/models/face-recognition/weights/{checkpoint_name}.ckpt"

print(f"\nSelected model: {model_name}")
print(f"Model checkpoint: {model_path}")

Available models:
1. sphere20
2. sphere36
3. sphere64
4. mobilenetv1
5. mobilenetv2
6. mobilenetv3_small
7. mobilenetv3_large

Selected model: mobilenetv3_large
Model checkpoint: ../../src/models/face-recognition/weights/mobilenetv3_large_5.ckpt


## Carregar Modelo

In [3]:
model_class = available_models[model_name]
model = model_class(embedding_dim=embedding_dim)

if model_path and os.path.exists(model_path):
    # Carregar checkpoint completo
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    # Verificar se é um checkpoint completo ou apenas pesos
    if 'model' in checkpoint:
        # É um checkpoint completo - extrair apenas o modelo
        model_state_dict = checkpoint['model']
        print(f"Loading complete checkpoint from epoch {checkpoint.get('epoch', 'unknown')}")
    else:
        # São apenas os pesos do modelo
        model_state_dict = checkpoint
        print("Loading model weights only")
    
    model.load_state_dict(model_state_dict)
    print(f"Model loaded from: {model_path}")
else:
    print(f"Warning: Checkpoint not found at {model_path}")
    print("Evaluating with random weights...")

model = model.to(device)
model.eval()
print("Model ready for evaluation")

Loading complete checkpoint from epoch 4
Model loaded from: ../../src/models/face-recognition/weights/mobilenetv3_large_5.ckpt
Model ready for evaluation


## Configurar Caminho do Dataset LFW

In [ ]:
lfw_dataset_path = "../../data/raw/lfw"  # Ajuste conforme necessário
threshold = 0.35  # Limiar de similaridade para classificação
save_metrics_path = "../../assets/evaluation_metrics"  # Pasta para salvar gráficos

# Criar diretório para métricas
os.makedirs(save_metrics_path, exist_ok=True)

print(f"LFW dataset path: {lfw_dataset_path}")
print(f"Similarity threshold: {threshold}")
print(f"Metrics save path: {save_metrics_path}")

if os.path.exists(lfw_dataset_path):
    print("✓ Path exists")
else:
    print("✗ Warning: Path does not exist!")

LFW dataset path: ../../data/raw/lfw
✓ Path exists


## Cálculo das Métricas no LFW

In [ ]:
print("Starting comprehensive evaluation on LFW dataset...")
print("="*70)

# Executar avaliação completa com todas as métricas
mean_similarity, predictions, metrics = eval(
    model,
    model_path=None,  # Modelo já carregado
    lfw_root=lfw_dataset_path,
    device=device,
    val_dataset='lfw',
    compute_full_metrics=True,  # Ativar cálculo completo de métricas
    save_metrics_path=save_metrics_path,  # Salvar gráficos
    threshold=threshold
)

print("\n" + "="*70)
print("EVALUATION SUMMARY")
print("="*70)

# Exibir métricas principais
print(f"\nBasic Metrics:")
print(f"  Mean Similarity: {metrics['mean_similarity']:.4f} ± {metrics['std_similarity']:.4f}")
print(f"  Precision:       {metrics['precision']:.4f}")
print(f"  Recall:          {metrics['recall']:.4f}")
print(f"  F1-Score:        {metrics['f1']:.4f}")
print(f"  Accuracy:        {metrics['accuracy']:.4f}")

# Exibir métricas ROC se disponíveis
if 'auc' in metrics:
    print(f"\nROC Metrics:")
    print(f"  AUC:             {metrics['auc']:.4f}")
    print(f"  EER:             {metrics['eer']:.4f} (threshold: {metrics['eer_threshold']:.4f})")
    
    # TAR@FAR metrics
    for key in metrics:
        if key.startswith('TAR@FAR'):
            far_value = key.split('=')[1]
            print(f"  TAR@FAR={far_value}: {metrics[key]:.4f}")

# Exibir matriz de confusão se disponível
if 'confusion_matrix' in metrics:
    print(f"\nConfusion Matrix:")
    print(f"  True Negatives:  {metrics['true_negatives']}")
    print(f"  False Positives: {metrics['false_positives']}")
    print(f"  False Negatives: {metrics['false_negatives']}")
    print(f"  True Positives:  {metrics['true_positives']}")
    print(f"\n  FAR (False Accept Rate): {metrics['far']:.4f}")
    print(f"  FRR (False Reject Rate): {metrics['frr']:.4f}")

print(f"\n{'='*70}")
print(f"Total pairs evaluated: {len(predictions)}")
print(f"Metrics and plots saved to: {save_metrics_path}")
print(f"  - ROC Curve: lfw_roc_curve.png")
print(f"  - Confusion Matrix: lfw_confusion_matrix.png")
print(f"{'='*70}")

Starting evaluation on LFW dataset...
LFW - Avaliacao Simplificada (Somente Pares Positivos):
Similaridade Media: 0.6256 | Desvio Padrao: 0.1339

Evaluation complete!
Average Similarity Score: 0.6256
LFW Accuracy: 0.9650
Total pairs evaluated: 3000


## Visualização dos Resultados

In [ ]:
# ROC Curve
if len(predictions) > 0:
    from IPython.display import Image, display
    
    plots_dir = '../../src/models/face-recognition/weights/evaluation_plots'
    roc_path = os.path.join(plots_dir, 'lfw_roc_curve.png')
    cm_path = os.path.join(plots_dir, 'lfw_confusion_matrix.png')
    
    print("📈 ROC Curve:")
    print("="*50)
    if os.path.exists(roc_path):
        display(Image(filename=roc_path))
    else:
        print("ROC curve not found. Make sure save_plots=True in eval().")
    
    print("\n📋 Confusion Matrix:")
    print("="*50)
    if os.path.exists(cm_path):
        display(Image(filename=cm_path))
    else:
        print("Confusion matrix not found. Make sure save_plots=True in eval().")
else:
    print("No predictions available for visualization.")

## Análise em Diferentes Tresholds

In [ ]:
if len(predictions) > 0:
    import matplotlib.pyplot as plt
    
    print("Analyzing threshold sensitivity...")
    
    # Testar range de thresholds
    test_thresholds = np.linspace(0.1, 0.9, 50)
    threshold_metrics = {
        'threshold': [],
        'accuracy': [],
        'f1': [],
        'precision': [],
        'recall': []
    }
    
    for thresh in test_thresholds:
        acc = eval_accuracy(predictions, thresh)
        f1 = compute_f1_score(predictions, thresh)
        prec = compute_precision(predictions, thresh)
        rec = compute_recall(predictions, thresh)
        
        threshold_metrics['threshold'].append(thresh)
        threshold_metrics['accuracy'].append(acc)
        threshold_metrics['f1'].append(f1)
        threshold_metrics['precision'].append(prec)
        threshold_metrics['recall'].append(rec)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(threshold_metrics['threshold'], threshold_metrics['accuracy'], 
            label='Accuracy', linewidth=2.5, marker='o', markersize=4)
    ax.plot(threshold_metrics['threshold'], threshold_metrics['f1'], 
            label='F1 Score', linewidth=2.5, marker='s', markersize=4)
    ax.plot(threshold_metrics['threshold'], threshold_metrics['precision'], 
            label='Precision', linewidth=2.5, marker='^', markersize=4)
    ax.plot(threshold_metrics['threshold'], threshold_metrics['recall'], 
            label='Recall', linewidth=2.5, marker='v', markersize=4)
    
    # Marcar best threshold
    best_threshold = metrics['best_threshold']
    ax.axvline(x=best_threshold, color='red', linestyle='--', 
              label=f'Best Threshold ({best_threshold:.3f})', linewidth=2.5)
    
    ax.set_xlabel('Threshold', fontsize=13, fontweight='bold')
    ax.set_ylabel('Score', fontsize=13, fontweight='bold')
    ax.set_title('Performance Metrics vs Threshold', fontsize=15, fontweight='bold')
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0.1, 0.9])
    ax.set_ylim([0, 1.05])
    
    plt.tight_layout()
    
    # Salvar
    threshold_plot_path = os.path.join(plots_dir, f'{model_name}_threshold_analysis.png')
    plt.savefig(threshold_plot_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Threshold analysis saved to: {threshold_plot_path}")
else:
    print("No predictions available for threshold analysis.")

## Tabela com Resumo Métricas

In [ ]:
if len(predictions) > 0:
    import pandas as pd
    
    # Extrair confusion matrix components
    cm = metrics['confusion_matrix']
    TP = cm[1,1]
    FP = cm[0,1]
    TN = cm[0,0]
    FN = cm[1,0]
    
    # Calcular derived metrics
    TAR = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    FAR = FP / (FP + TN) if (FP + TN) > 0 else 0.0
    FRR = FN / (FN + TP) if (FN + TP) > 0 else 0.0
    
    # Criar tabela
    metrics_table = {
        'Metric': [
            'Mean Similarity',
            'Std Similarity',
            '---',
            'Best Threshold',
            'Accuracy',
            'F1 Score',
            'Precision',
            'Recall',
            'AUC Score',
            '---',
            'TAR (True Accept Rate)',
            'FAR (False Accept Rate)',
            'FRR (False Reject Rate)',
            '---',
            'True Negatives (TN)',
            'False Positives (FP)',
            'False Negatives (FN)',
            'True Positives (TP)',
            '---',
            'Total Pairs',
            'Positive Pairs'
        ],
        'Value': [
            f"{metrics['mean_similarity']:.4f}",
            f"{metrics['std_similarity']:.4f}",
            '',
            f"{metrics['best_threshold']:.4f}",
            f"{metrics['accuracy']:.4f}",
            f"{metrics['f1_score']:.4f}",
            f"{metrics['precision']:.4f}",
            f"{metrics['recall']:.4f}",
            f"{metrics['auc_score']:.4f}",
            '',
            f"{TAR:.4f}",
            f"{FAR:.4f}",
            f"{FRR:.4f}",
            '',
            f"{TN}",
            f"{FP}",
            f"{FN}",
            f"{TP}",
            '',
            f"{len(predictions)}",
            f"{int(np.sum(predictions[:, 3].astype(int)))}"
        ]
    }
    
    df_metrics = pd.DataFrame(metrics_table)
    
    print("="*60)
    print(f"EVALUATION SUMMARY - {model_name.upper()}")
    print("="*60)
    print(df_metrics.to_string(index=False))
    print("="*60)
    
    # Salvar CSV
    csv_path = os.path.join(plots_dir, f'{model_name}_metrics_summary.csv')
    df_metrics.to_csv(csv_path, index=False)
    print(f"\n✅ Metrics summary saved to: {csv_path}")
    
else:
    print("No predictions available for metrics summary.")

## Exportar Json Completo

In [ ]:
## Exportar Resultados Completos em JSON

if len(predictions) > 0:
    import json
    
    # Extrair confusion matrix
    cm = metrics['confusion_matrix']
    TP, FP, TN, FN = cm[1,1], cm[0,1], cm[0,0], cm[1,0]
    
    # Calcular derived metrics
    TAR = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    FAR = FP / (FP + TN) if (FP + TN) > 0 else 0.0
    FRR = FN / (FN + TP) if (FN + TP) > 0 else 0.0
    
    # Criar estrutura completa
    evaluation_results = {
        'model_info': {
            'model_name': model_name,
            'checkpoint': checkpoint_name,
            'embedding_dim': embedding_dim,
            'checkpoint_path': model_path
        },
        'dataset_info': {
            'dataset': 'LFW',
            'dataset_path': lfw_dataset_path,
            'num_pairs': int(len(predictions)),
            'num_positive_pairs': int(np.sum(predictions[:, 3].astype(int))),
            'num_negative_pairs': int(len(predictions) - np.sum(predictions[:, 3].astype(int)))
        },
        'similarity_metrics': {
            'mean': float(metrics['mean_similarity']),
            'std': float(metrics['std_similarity']),
            'min': float(np.min(predictions[:, 2].astype(float))),
            'max': float(np.max(predictions[:, 2].astype(float))),
            'median': float(np.median(predictions[:, 2].astype(float)))
        },
        'classification_metrics': {
            'best_threshold': float(metrics['best_threshold']),
            'accuracy': float(metrics['accuracy']),
            'f1_score': float(metrics['f1_score']),
            'precision': float(metrics['precision']),
            'recall': float(metrics['recall']),
            'auc_score': float(metrics['auc_score'])
        },
        'biometric_metrics': {
            'TAR': float(TAR),
            'FAR': float(FAR),
            'FRR': float(FRR)
        },
        'confusion_matrix': {
            'true_negatives': int(TN),
            'false_positives': int(FP),
            'false_negatives': int(FN),
            'true_positives': int(TP)
        },
        'generated_files': {
            'roc_curve': os.path.join(plots_dir, 'lfw_roc_curve.png'),
            'confusion_matrix': os.path.join(plots_dir, 'lfw_confusion_matrix.png'),
            'threshold_analysis': os.path.join(plots_dir, f'{model_name}_threshold_analysis.png'),
            'metrics_csv': os.path.join(plots_dir, f'{model_name}_metrics_summary.csv')
        }
    }
    
    # Salvar JSON
    json_path = os.path.join(plots_dir, f'{model_name}_evaluation_results.json')
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(evaluation_results, f, indent=2, ensure_ascii=False)
    
    print("="*70)
    print("📁 EVALUATION FILES GENERATED")
    print("="*70)
    print(f"✅ JSON Results:      {json_path}")
    print(f"✅ CSV Summary:       {csv_path}")
    print(f"✅ ROC Curve:         {roc_path}")
    print(f"✅ Confusion Matrix:  {cm_path}")
    print(f"✅ Threshold Plot:    {threshold_plot_path}")
    print("="*70)
    
    print("\n✅ All evaluation metrics calculated and exported successfully!")
else:
    print("No predictions available for export.")

## Análise do Resultado

In [6]:
if len(predictions) > 0:
    similarities = predictions[:, 2].astype(float)
    
    # Estatísticas
    print("Similarity Statistics:")
    print(f"Mean: {np.mean(similarities):.4f}")
    print(f"Std: {np.std(similarities):.4f}")
    print(f"Min: {np.min(similarities):.4f}")
    print(f"Max: {np.max(similarities):.4f}")
    print(f"Median: {np.median(similarities):.4f}")
else:
    print("No predictions to analyze.")

Similarity Statistics:
Mean: 0.6256
Std: 0.1339
Min: 0.1134
Max: 0.9901
Median: 0.6407


## Avaliar Vários Modelos (Batch Evaluation)

In [ ]:
# Lista de modelos para avaliar
models_to_evaluate = [
    ("mobilenetv3_large", "../../src/models/face-recognition/weights/mobilenetv3_large_5.ckpt"),
    # Adicione mais modelos aqui
    # ("sphere20", "../../src/models/face-recognition/weights/sphere20_mcp.ckpt"),
    # ("mobilenetv2", "../../src/models/face-recognition/weights/mobilenetv2_mcp.ckpt"),
]

results = []
print("\nBatch Evaluation of Multiple Models")
print("="*70)

for model_name, checkpoint_path in models_to_evaluate:
    if not os.path.exists(checkpoint_path):
        print(f"Skipping {model_name}: checkpoint not found at {checkpoint_path}")
        continue
    
    print(f"\nEvaluating {model_name}...")
    
    # Inicializar modelo
    model_class = available_models[model_name]
    model_eval = model_class(embedding_dim=512).to(device)
    
    # Carregar checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    if 'model' in checkpoint:
        model_eval.load_state_dict(checkpoint['model'])
    else:
        model_eval.load_state_dict(checkpoint)
    
    # Avaliar
    score, preds, m = eval(
        model_eval,
        model_path=None,
        lfw_root=lfw_dataset_path,
        device=device,
        compute_full_metrics=False,  # Mais rápido para comparação
        threshold=threshold
    )
    
    results.append({
        'model': model_name,
        'mean_similarity': score,
        'precision': m['precision'],
        'recall': m['recall'],
        'f1': m['f1'],
        'accuracy': m['accuracy'],
        'num_pairs': len(preds)
    })

print("\n" + "="*70)
print("COMPARISON SUMMARY")
print("="*70)

if results:
    print(f"\n{'Model':<20} {'Similarity':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'Accuracy':<12}")
    print("-" * 90)
    
    for result in results:
        print(f"{result['model']:<20} {result['mean_similarity']:<12.4f} "
              f"{result['precision']:<12.4f} {result['recall']:<12.4f} "
              f"{result['f1']:<12.4f} {result['accuracy']:<12.4f}")
else:
    print("No models evaluated.")

## Inspeção de Predição de Samples

In [ ]:
if len(predictions) > 0:
    print("\nSample Predictions:")
    print("="*70)
    
    # Mostrar alguns pares positivos
    positive_pairs = predictions[predictions[:, 3] == '1']
    print(f"\n{'-'*70}")
    print("POSITIVE PAIRS (Same Person) - First 3:")
    print(f"{'-'*70}")
    for i in range(min(3, len(positive_pairs))):
        path1, path2, similarity, gt = positive_pairs[i]
        print(f"\nPair {i+1}:")
        print(f"  Image 1:    {os.path.basename(path1)}")
        print(f"  Image 2:    {os.path.basename(path2)}")
        print(f"  Similarity: {float(similarity):.4f}")
        print(f"  Prediction: {'✓ SAME' if float(similarity) > threshold else '✗ DIFFERENT'}")
    
    # Mostrar alguns pares negativos se existirem
    negative_pairs = predictions[predictions[:, 3] == '0']
    if len(negative_pairs) > 0:
        print(f"\n{'-'*70}")
        print("NEGATIVE PAIRS (Different Person) - First 3:")
        print(f"{'-'*70}")
        for i in range(min(3, len(negative_pairs))):
            path1, path2, similarity, gt = negative_pairs[i]
            print(f"\nPair {i+1}:")
            print(f"  Image 1:    {os.path.basename(path1)}")
            print(f"  Image 2:    {os.path.basename(path2)}")
            print(f"  Similarity: {float(similarity):.4f}")
            print(f"  Prediction: {'✗ SAME' if float(similarity) > threshold else '✓ DIFFERENT'}")
    
    print(f"\n{'='*70}")
else:
    print("No predictions to display.")